In [ ]:
import sqlite3
from typing import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.types import interrupt
from langchain_ollama import ChatOllama
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_community.tools import DuckDuckGoSearchRun
from langgraph.checkpoint.sqlite import SqliteSaver

# ===================== MODEL =====================
model = ChatOllama(
    model="qwen2.5:0.5b",
    temperature=0.3
)

# ===================== STATE =====================
class PostState(TypedDict):
    topic: str
    search: str
    post: str
    suggestion: str
    approved: bool
    iteration: int
    max_iteration: int

# ===================== TOOLS =====================
search_tool = DuckDuckGoSearchRun(name="Search")

# ===================== NODES =====================
def search_topic(state: PostState):
    topic = state["topic"]
    queries = {
        "Recent Developments": f"Latest news, updates, statistics, and reforms related to {topic}",
        "Historical Context": f"Brief history and evolution of {topic}",
        "Current Landscape": f"Current state, challenges, and adoption of {topic}",
        "Future Outlook": f"Future trends, roadmap, and expected impact of {topic}",
    }

    collected_results = []

    for section, query in queries.items():
        try:
            result = search_tool.run(query)
            if result:
                collected_results.append(f"{section}:\n{result}")
        except Exception:
            continue  # Fail-safe: search issues should not break the workflow

    search_context = "\n\n".join(collected_results)

    return {
        "search": search_context.strip()
    }


def generate_post(state: PostState):
    prompt = [
        SystemMessage(
            content="""
                You are a professional AI/ML & GenAI Engineer with strong technical and communication skills.
                You write high-quality, original LinkedIn posts for a professional audience including
                software engineers, AI practitioners, and tech leaders.
                
                Your writing style is:
                - Clear, concise, and professional
                - Informative but not academic
                - Confident but not promotional
                - Easy to skim (short paragraphs, logical flow)
                """
            ),
        HumanMessage(
            content=f"""
                TASK:
                Write a professional LinkedIn post on the topic below.
                
                TOPIC:
                {state['topic']}
                
                REFERENCE INFORMATION:
                Use the following research material strictly as context and factual grounding.
                Do NOT copy text verbatim.
                Summarize, synthesize, and rephrase in your own words.
                
                {state['search']}

                USER SUGGESTION:
                Incorporate the following guidance if relevant and appropriate:
                
                {state['suggestion']}
                
                WRITING GUIDELINES:
                - Start with a strong opening hook (1–2 lines)
                - Clearly explain why the topic matters today
                - Highlight 2–3 key insights or developments
                - Keep the tone professional and practical
                - Avoid emojis, hashtags, or excessive formatting
                - Length: 120–180 words
                - Do NOT mention being an AI or language model
                
                OUTPUT:
                Return only the final LinkedIn post content.
                """
            )
        ]

    post = model.invoke(prompt).content

    return {
        "post": post,
        "approved": False,
        "iteration": state["iteration"] + 1
    }


def human_review(state: PostState):
    print("\n" + "="*70)
    print(f"👤 HUMAN REVIEW REQUIRED - Iteration: {state['iteration']}/{state['max_iteration']}")
    print("Topic:", state["topic"])
    print("\nPROPOSED POST:\n")
    print(state["post"])
    print("\n" + "="*70)
    print("\n"*3)
    
    while True:
        if state['iteration'] >= state['max_iteration']:
            print("Your approval limit exceeded.")
            break
        
        decision = input("Approve this post? (yes / no / feedback): ").strip().lower()
        
        if decision in ["yes", "y"]:
            return {"approved": True}
            
        elif decision in ["no", "n"]:
            return {"approved": False}

        elif decision in ["feedback", "f"]:
            suggestion = input("Kindly enter your suggestion so that I can improve it more better way: ")
            return {"approved": False, 'suggestion': suggestion}
    
    return {"approved": True}
    

def check_condition(state: PostState):
    if state["approved"]:
        return "finalize_post"
    if state["iteration"] >= state["max_iteration"]:
        return "finalize_post"
    return "generate_post"


def finalize_post(state: PostState):
    print("\n✅ Finalizing output. Ready for LinkedIn.")
    print("\n" + "="*70)
    print("Topic:", state["topic"])
    print("\nPROPOSED POST:\n")
    print(state["post"])
    print("\n" + "="*70)
    return state

# ===================== GRAPH =====================
graph = StateGraph(PostState)

graph.add_node("search_topic", search_topic)
graph.add_node("generate_post", generate_post)
graph.add_node("human_review", human_review)
graph.add_node("finalize_post", finalize_post)

graph.add_edge(START, "search_topic")
graph.add_edge("search_topic", "generate_post")
graph.add_edge("generate_post", "human_review")

graph.add_conditional_edges(
    "human_review",
    check_condition,
    {
        "generate_post": "generate_post",
        "finalize_post": "finalize_post",
    }
)

graph.add_edge("finalize_post", END)

# ===================== CHECKPOINTER =====================
conn = sqlite3.connect("LinkedIn_Post.db", check_same_thread=False)
checkpointer = SqliteSaver(conn)

workflow = graph.compile(checkpointer=checkpointer)


# ===================== RUN (JUPYTER) =====================
config = {
    "configurable": {
        "thread_id": "post-1"
    }
}

initial_state = {
    "topic": "Era of Aritificial Intelligence",
    "search": "",
    "suggestion": "",
    "post": "",
    "approved": False,
    "iteration": 0,
    "max_iteration": 4,
}

response = workflow.invoke(initial_state, config=config)
# print(response['post'])


